In [18]:
import pandas as pd #基础包
import numpy as np #基础包
import matplotlib as mpl
import matplotlib.dates as mdate 
import matplotlib.pyplot as plt 
from matplotlib.colors import ListedColormap
import seaborn as sns 
from sklearn.datasets import fetch_olivetti_faces #人脸数据库
from sklearn.metrics import roc_auc_score,roc_curve,auc
import re
import jieba
import pandas as pd #基础包
import numpy as np #基础包
import matplotlib as mpl
import matplotlib.dates as mdate 
import matplotlib.pyplot as plt 
from matplotlib.colors import ListedColormap
import seaborn as sns 
from sklearn.datasets import fetch_olivetti_faces #人脸数据库
from sklearn.metrics import roc_auc_score,roc_curve,auc
#from transformers.convert_graph_to_onnx import *
douyin = pd.read_csv("douyin.csv",encoding="utf-8",index_col = 0)
stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
#stopwords = [line.strip() for line in open('F:\\2023暑期班材料\\stopwords0.txt',encoding='UTF-8').readlines()]

In [19]:
douyin.head()
douyin['script'].shape
douyin['script']

0              # 为中国点赞👍，为我们医务工作者点赞👍，#抗疫#抖音小助手 #穿越时空只想见你
1       #2020 盘点2020那些事儿，“20个数字穿越2020” #抗疫  #火神山  #嫦娥五号
2                                #2020上海宝马工程机械展 上海加油#抗疫
3            #4月4日全国哀悼 柘木所全体民辅警哀悼#抗疫 烈士和逝世同胞@平安湖北 @荆州公安
4              #NZ#lockdown#stayhome 老外写中文 #抗疫#快乐宅家同心抗疫
                             ...                       
4021    🎉🎉恭喜#竹溪 县#妇幼保健院 #抗疫 #天使 👼🏻黄霞，#荣获 #十堰市 五一劳动奖章 🎉🎉
4022                          🎨《这是一个画画的文案》#抗疫应援 #抗疫 #手绘
4023                      🐴住！冬季抗疫宝典👍🏻#抗疫 #防护措施 @DOU+小助手
4024         👏会跳舞的抗疫医护宝宝来报道啦，😘大家想看看跳舞的样子吗？有请天使宝宝出场❤️#抗疫
4025              😈三条尾巴的老鼠见过嘛🐭#本命 #新冠病毒 #抗疫 #trantision
Name: script, Length: 4026, dtype: object

In [20]:
#一个清理文本的函数
def clean_text(text):
    text  = text.replace('\n'," ")
    text = re.sub('-'," ",text)
    text = re.sub(' ',"",text)
    text = re.sub(r"\d+/\d+/\d+", "", text) #日期，对主体模型没什么意义
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) #时间，没意义
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #邮件地址，没意义
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text) #网址，没意义
    pure_text = ''
    for letter in text:
        if letter.isalpha() or letter ==' ':
            pure_text += letter
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    return text#可以自己处理内容

#docs是清理之后的一句一句话
docs = douyin["script"]
docs = docs.apply(lambda x :clean_text(x))#对docs的每个元素执行clean_text函数
docs[0]
#将一句一句话分割成词
texts = []
for doc in docs: 
    doc = str(doc)
    #text = jieba.lcut(doc)
    text = jieba.lcut(doc)
    text = [w for w in text if w not in stopwords]
    texts.append(list(text))
texts[0]


['中国', '点赞', '医务', '工作者', '点赞', '抗疫', '穿越时空', '想见']

In [ ]:
dictionary =corpora.Dictionary(texts) #将text编成字典的形式储存
corpus = [dictionary.doc2bow(text) for text in texts]
print("字典的第一个词dictionary[0]：",dictionary[0])
print("文本数据中的第一句话docs[0]：",docs[0])
print("第一句话分词后的结果texts[0]：",texts[0])

#左边的数字代表词典中的某个词（比如0代表abandon，999代表zoo），
#右边的数字代表这个词在文档中出现的频数，
#（0，0）代表abandon这个词在文档中出现了0次，
#（999, 2）代表zoo这个词在文档中出现了2次
print("第一句话语料化后的结果corpus[0]：",corpus[0])
#print("第二句话语料化后的结果corpus[1]：",corpus[1])

#注意，texts和corpus的数目是不对应的，因为texts包括重复词，而corpus不包括
print(len(texts[0]),len(corpus[0]))
#tf_idf作为一种文本的加权手段，突出词汇的重要性
#tf_idf = word_freq * log（全部文本数 / 包含该词的文本数）



In [ ]:
tf_idf_model = TfidfModel(corpus, normalize=False)

corpus_tfidf = models.TfidfModel(corpus)#计算结果

word_tf_tdf = list(tf_idf_model[corpus])#将结果存成列表

df_tf_idf = DataFrame(word_tf_tdf)#将列表存成数据框

#print('词典:', dictionary.token2id)
#print('词频:', corpus)
#print('词的tf-idf值:', word_tf_tdf)
print("第一句话分词结果texts"+ str(texts[0])+"\n" +"第一句话语料化后的结果corpus"+str(corpus[0])+"\n" +"第一句话tf-idf后的结果word_tf_tdf"+str(word_tf_tdf[0]))
#print(corpus_tfidf)

情感分析

In [ ]:
#计算积极词汇的平均向量
pos = ["点赞","加油","开心","快乐","不败","不朽","渡过难关","致敬"]
neg = ["哀悼","不幸","放弃","惨烈","恶意中伤","抢救无效","去世","天堂","病痛","默哀","确诊","伤心"]
k1 = model.wv[pos[0]]
for i in range(1,len(pos)):
    k1 = k1 + model.wv[pos[i]]
num_pos = len(pos)
k_pos = k1/num_pos #计算所有积极词汇向量的均值

#计算消极词汇的平均向量
k2 = model.wv[neg[0]]
for i in range(1,len(neg)):
    k2 = k2 + model.wv[neg[i]]
num_neg = len(neg)
k_neg = k2/num_neg #计算所有消极词汇向量的均值
import numpy as np

# 初始化一个空列表来存储每个词语的情感得分
emotion_scores = []

# 遍历所有词语，计算情感得分并添加到列表中
for word in words:
    score = emo(word)[1]  # 假设emo函数返回的是一个情感得分
    emotion_scores.append(score)

# 将列表转换为NumPy数组，以便于计算平均值
emotion_scores_array = np.array(emotion_scores)

# 计算情感得分的平均值
average_emotion_score = np.mean(emotion_scores_array)

# 打印平均情感得分
print("Average Emotion Score:", average_emotion_score)
#一个用于计算词语情感得分的函数
def emo(w):
    set_seed = 888
    word = model.wv[w]
    direction1 = k_pos 
    direction2 = -k_neg 
    dian1 = np.dot(word,direction1) #词语在积极
    dian2 = np.dot(word,direction2) #消极这个维度上的投影
    sim1_result = dian1/(np.linalg.norm(word)*np.linalg.norm(direction1))
    sim2_result = dian2/(np.linalg.norm(word)*np.linalg.norm(direction2))
    result1 = "【" + str(w) + "】" + "积极情感得分为：" + str(sim1_result) 
    result2 = "【" + str(w) + "】" + "消极情感得分为：" + str(sim2_result) 
    return result1,result2,sim1_result,sim2_result
print(str(emo("胜利")[0])+ "\n" + str(emo("胜利")[1]))
print(str(emo("疫情")[0])+ "\n" + str(emo("疫情")[1]))
print(str(emo("美国")[0])+ "\n" + str(emo("美国")[1]))
print(str(emo("美国佬")[0])+ "\n" + str(emo("美国佬")[1]))
print(str(emo("医生")[0])+ "\n" + str(emo("医生")[1]))
print(str(emo("死者")[0])+ "\n" + str(emo("死者")[1]))